## Deep Deterministic Policy Gradient (DDPG)

In [ ]:
%%capture

!apt-get install -y xvfb

!pip install gym==0.23 \
    pytorch-lightning==1.6.0 \
    pyvirtualdisplay \
    brax==0.10.5

#### Setup virtual display

In [ ]:
from pyvirtualdisplay import Display
Display(visible=False, size=(1400, 900)).start()

#### Import the necessary code libraries

In [ ]:
import copy
import gym
import torch
import random
import functools

import numpy as np
import torch.nn.functional as F

from collections import deque, namedtuple
from IPython.display import HTML
from base64 import b64encode

from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
from torch.optim import AdamW

from pytorch_lightning import LightningModule, Trainer

import brax
from brax import envs
from brax.envs.wrappers import gym as gym_wrapper
from brax.envs.wrappers import torch as torch_wrapper
from brax.io import html

device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

In [ ]:
def display_video(episode=0):
  video_file = open(f'/content/videos/rl-video-episode-{episode}.mp4', "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"<video width=600 controls><source src='{video_url}'></video>")

In [ ]:
def create_environment(env_name, num_envs=256, episode_length=1000):
    env = envs.create(env_name, batch_size=num_envs, episode_length=episode_length, backend='spring')
    env = gym_wrapper.VectorGymWrapper(env)
    env = torch_wrapper.TorchWrapper(env, device=device)
    return env

In [ ]:
@torch.no_grad()
def test_env(env_name, policy=None):
  env = envs.create(env_name, episode_length=1000, backend='spring')
  env = gym_wrapper.GymWrapper(env)
  env = torch_wrapper.TorchWrapper(env, device=device)
  ps_array = []
  state = env.reset()
  for i in range(1000):
    if policy:
      action = algo.policy.net(state.unsqueeze(0)).squeeze()
    else:
      action = torch.from_numpy(env.action_space.sample()).to(device)
    state, _, _, _ = env.step(action)
    ps_array.extend([env.unwrapped._state.pipeline_state]*5)
  return HTML(html.render(env.unwrapped._env.sys, ps_array))

In [ ]:
test_env('ant')

#### Create the gradient policy

#### Create the Deep Q-Network

In [ ]:
# Start tensorboard.
!rm -r /content/lightning_logs/
!rm -r /content/videos/
%load_ext tensorboard
%tensorboard --logdir /content/lightning_logs/